## CIFAR-10 : test of different CNNs to find the best performance on test data

In [1]:
from tensorflow import keras             
from tensorflow.keras.models import Sequential ,load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam ,SGD,RMSprop,Adagrad,Adadelta,Adamax,Nadam
from tensorflow.keras import utils
from tensorflow.keras.datasets import cifar10
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np 

import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = "notebook_connected"

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [2]:
(x_train10, y_train10), (x_test10, y_test10) = cifar10.load_data()

In [3]:
# 10 classes of images:
classes = ['airplane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truk']

In [4]:
# Lets see the examples of images of the classes
fig = make_subplots(rows=1, cols=10)
for i in range(10):
  label_indexes = np.where(y_train10==i)[0] 
  index = np.random.choice(label_indexes) 
  img = x_train10[index] 
  fig.add_trace(go.Image(z=img),row=1,col=i+1)

fig.update_layout(
                  width = 1300, height = 300,
   
                 showlegend=False
)
fig.show()


In [5]:
#Scaling input images
x_train_norm = x_train10.astype('float32') 
x_train_norm = x_train_norm / x_train10.max() 
x_test_norm = x_test10.astype('float32') 
x_test_norm = x_test_norm / x_test10.max()

In [6]:
#Transform labels to categorical
y_train10 = utils.to_categorical(y_train10, 10)
y_test10 = utils.to_categorical(y_test10, 10)

In [7]:
#Check the shapes of train/test datas
print(x_train_norm.shape)
print(x_test_norm.shape)
print(y_train10.shape)
print(y_test10.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)
(50000, 10)
(10000, 10)


In [8]:
# Some useful callbacs for dynamically displaying the process of training and early stopping
class acc_mon(keras.callbacks.Callback):
    def __init__(self,f,e_stop=20,a_stop=0.60):
        self.x=[]
        self.y1=[]
        self.y2=[]
        self.fw=f
        self.e_stop=e_stop
        self.a_stop=a_stop
    def on_train_begin(self, logs=None):
        print("Start training:")
        self.fw.add_scatter(x=self.x,y=self.y1,mode = 'lines',name = 'Train accuracy', line_color="green", showlegend = True)
        self.fw.add_scatter(x=self.x,y=self.y2,mode = 'lines',name = 'Validation accuracy', line_color="red", showlegend = True)
        self.fw.update_layout(title_text="Train and validation accuracy",
                  title_x = 0.45,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom",template='plotly_dark',
                  xaxis_title='Epoch', yaxis_title='Accuracy',width = 800, height = 500)
        scat1=self.fw.data[0]
        scat1.y=self.y1
        scat1.x=self.x
        scat2=self.fw.data[1]
        scat2.y=self.y2
        scat2.x=self.x
    def on_epoch_end(self, epoch, logs):
        if epoch > self.e_stop and logs.get('val_accuracy') < self.a_stop:
            print('Poor performance! Rebuild model!')
            print('-> epoch:',epoch,'acc:',round(logs['accuracy'],6),'val_acc:',round(logs['val_accuracy'],6))
            self.model.stop_training = True
        elif epoch > 0 and epoch % 10 == 0:
            print('-> epoch:',epoch,' ',end='')
            for i in logs.keys():
                print(i,":",round(logs[i],8),' ',end='')
            print('')
        print('.', end='')
        self.x.append(epoch)
        self.y1.append(logs['accuracy'])
        self.y2.append(logs['val_accuracy'])
        scat1=self.fw.data[0]
        scat1.y=self.y1
        scat1.x=self.x
        scat2=self.fw.data[1]
        scat2.y=self.y2
        scat2.x=self.x
       

In [9]:

class train_mon(keras.callbacks.Callback):
    def __init__(self,f):
        self.x=[]
        self.y1=[]
        self.y2=[]
        self.fw=f
    def on_train_begin(self, logs=None):
        print("Start training:")
        self.fw.add_scatter(x=self.x,y=self.y1,mode = 'lines',name = 'Train error', line_color="green", showlegend = True)
        self.fw.add_scatter(x=self.x,y=self.y2,mode = 'lines',name = 'Validation error', line_color="red", showlegend = True)
        self.fw.update_layout(title_text="Train and validation errors",
                  title_x = 0.45,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom",template='gridon',
                  xaxis_title='Epoch', yaxis_title='Error',width = 800, height = 500)
        scat1=self.fw.data[0]
        scat1.y=self.y1
        scat1.x=self.x
        scat2=self.fw.data[1]
        scat2.y=self.y2
        scat2.x=self.x
    def on_epoch_end(self, epoch, logs):
        if epoch > 0 and epoch % 10 == 0:
            print('-> epoch:',epoch,' ',end='')
            for i in logs.keys():
                print(i,":",round(logs[i],8),' ',end='')
            print('')
        print('.', end='')
        self.x.append(epoch)
        self.y1.append(logs['loss'])
        self.y2.append(logs['val_loss'])
        scat1=self.fw.data[0]
        scat1.y=self.y1
        scat1.x=self.x
        scat2=self.fw.data[1]
        scat2.y=self.y2
        scat2.x=self.x
 

In [10]:
# Early stop of training model when it shows overfiting condition
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=50,restore_best_weights=True)

In [11]:
# The callback to reduce the learning rate on plateau
learn_rate=keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=15)

In [12]:
# Data Augmentation to expand training dataset
datagen = ImageDataGenerator(
         rotation_range=20,
         width_shift_range=0.2,
         height_shift_range=0.2,
         horizontal_flip=True)
datagen.fit(x_train_norm)


In [13]:
batch_size = 128

### Let's construct a CNN model with performance over 90% on test data

In [13]:
# CIFAR-10 CNN (test_acc=90,39%) 
def createModel(N1=128,N2=256,N3=512,N4=768,act_f='relu'):
    model = Sequential()
    model.add(BatchNormalization(input_shape=(32, 32, 3)))
    model.add(Conv2D(N1, (3, 3), padding='same',activation=act_f))
    model.add(BatchNormalization())
    model.add(Conv2D(N2, (3, 3), padding='same', activation=act_f))
    model.add(BatchNormalization())
    model.add(Conv2D(N3, (3, 3), padding='same',activation=act_f))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(BatchNormalization())
    model.add(Conv2D(N4, (3, 3), padding='same',activation=act_f))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Conv2D(N4, (3, 3), padding='same', activation=act_f))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Conv2D(N4, (3, 3), padding='same', activation=act_f))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
   
    model.add(Dense(768, activation=act_f))
    model.add(Dense(192, activation=act_f))
    model.add(Dense(48, activation=act_f))
    model.add(Dropout(0.10))
    
    model.add(Dense(10, activation='softmax'))

    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

In [14]:
model = createModel(act_f='relu')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 32, 32, 3)         12        
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 128)       3584      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 128)       512       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 256)       295168    
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 256)       1024      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 512)       1180160   
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 512)       0

In [15]:
fw = go.FigureWidget()
fw

FigureWidget({
    'data': [], 'layout': {'template': '...'}
})

In [16]:
#Training and results for constructed model on CIFAR-10 dataset
batch_size=100
callbacks_list=[learn_rate,early_stop,acc_mon(fw)]
report=model.fit(datagen.flow(x_train_norm, y_train10, batch_size=batch_size),\
                    steps_per_epoch=x_train_norm.shape[0] // batch_size,epochs=1000,\
                    verbose=0,validation_data=(x_test_norm,y_test10),callbacks=callbacks_list)


Start training:
..........-> epoch: 10  loss : 1.89399481  accuracy : 0.26050001  val_loss : 1.7807951  val_accuracy : 0.31659999  lr : 0.001  
..........-> epoch: 20  loss : 0.56659812  accuracy : 0.81588  val_loss : 0.57448608  val_accuracy : 0.82480001  lr : 0.001  
..........-> epoch: 30  loss : 0.33778247  accuracy : 0.89073998  val_loss : 0.45391279  val_accuracy : 0.8599  lr : 0.001  
..........-> epoch: 40  loss : 0.23873527  accuracy : 0.92306  val_loss : 0.4547233  val_accuracy : 0.87980002  lr : 0.001  
..........-> epoch: 50  loss : 0.14460276  accuracy : 0.95252001  val_loss : 0.38944286  val_accuracy : 0.898  lr : 1e-04  
..........-> epoch: 60  loss : 0.11068387  accuracy : 0.96498001  val_loss : 0.38216534  val_accuracy : 0.90469998  lr : 1e-04  
..........-> epoch: 70  loss : 0.09461119  accuracy : 0.97056001  val_loss : 0.39659065  val_accuracy : 0.90869999  lr : 1e-04  
..........-> epoch: 80  loss : 0.08624506  accuracy : 0.9727  val_loss : 0.39555153  val_accuracy 

In [17]:
df_report = pd.DataFrame(report.history) 
df_report['epoch'] = report.epoch
df_report.tail()

,loss,accuracy,val_loss,val_accuracy,lr,epoch
103,0.084221,0.97180,0.400650,0.9088,1.000000e-07,103
104,0.083706,0.97202,0.399978,0.9092,1.000000e-07,104
105,0.084562,0.97302,0.400265,0.9092,1.000000e-07,105
106,0.084583,0.97224,0.398516,0.9094,1.000000e-07,106
107,0.083721,0.97194,0.398672,0.9091,1.000000e-07,107


In [18]:
#Testing on test data
test_loss, test_acc = model.evaluate(x_test_norm,  y_test10, verbose=0)
print('Accuracy on test data :', test_acc)

Accuracy on test data : 0.9039999842643738


### Let's construct neural nets using different pretrained models that won Imagenet competitions in the past

In [21]:
# DenseNet201
imp_net = keras.applications.DenseNet201(weights='imagenet',include_top=False,
            input_shape=(32, 32, 3))

In [22]:
model_dn201 = Sequential()
model_dn201.add(imp_net)
model_dn201.add(Dropout(0.25))
model_dn201.add(Flatten())
model_dn201.add(Dense(960, activation='relu'))
model_dn201.add(Dense(120, activation='relu'))
model_dn201.add(Dense(30, activation='relu'))
model_dn201.add(Dropout(0.10))
model_dn201.add(Dense(10, activation='softmax'))
model_dn201.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model_dn201.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet201 (Functional)     (None, 1, 1, 1920)        18321984  
_________________________________________________________________
dropout (Dropout)            (None, 1, 1, 1920)        0         
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 960)               1844160   
_________________________________________________________________
dense_1 (Dense)              (None, 120)               115320    
_________________________________________________________________
dense_2 (Dense)              (None, 30)                3630      
_________________________________________________________________
dropout_1 (Dropout)          (None, 30)                0

In [23]:
print(len(model_dn201.trainable_weights))
imp_net.trainable = False #Freezing weights of DenseNet201
print(len(model_dn201.trainable_weights))

610
8


In [24]:
fw1 = go.FigureWidget()
fw1

FigureWidget({
    'data': [], 'layout': {'template': '...'}
})

In [25]:
batch_size=128
callbacks_list=[learn_rate,early_stop,train_mon(fw1)]
report=model_dn201.fit(datagen.flow(x_train_norm, y_train10, batch_size=batch_size),\
                    steps_per_epoch=x_train_norm.shape[0] // batch_size,epochs=1000,\
                    verbose=0,validation_data=(x_test_norm,y_test10),callbacks=callbacks_list)


Start training:
..........-> epoch: 10  loss : 0.58702987  accuracy : 0.80453962  val_loss : 0.86608195  val_accuracy : 0.72329998  lr : 0.001  
..........-> epoch: 20  loss : 0.44816661  accuracy : 0.85186076  val_loss : 0.77058572  val_accuracy : 0.75749999  lr : 0.001  
..........-> epoch: 30  loss : 0.37114835  accuracy : 0.87626326  val_loss : 0.48411754  val_accuracy : 0.83819997  lr : 0.001  
..........-> epoch: 40  loss : 0.31819764  accuracy : 0.89140201  val_loss : 0.4561725  val_accuracy : 0.85439998  lr : 0.001  
..........-> epoch: 50  loss : 0.28624251  accuracy : 0.90433508  val_loss : 0.60125166  val_accuracy : 0.82169998  lr : 0.001  
..........-> epoch: 60  loss : 0.15341526  accuracy : 0.94886911  val_loss : 0.42246261  val_accuracy : 0.88349998  lr : 1e-04  
..........-> epoch: 70  loss : 0.12295455  accuracy : 0.95855391  val_loss : 0.44570455  val_accuracy : 0.88230002  lr : 1e-04  
..........-> epoch: 80  loss : 0.1094804  accuracy : 0.96268445  val_loss : 0.4308

In [26]:
df_report = pd.DataFrame(report.history) 
df_report['epoch'] = report.epoch
df_report.tail()

,loss,accuracy,val_loss,val_accuracy,lr,epoch
104,0.105933,0.963988,0.434345,0.8877,1.000000e-07,104
105,0.107527,0.963908,0.435862,0.8879,1.000000e-07,105
106,0.107336,0.962905,0.436773,0.8883,1.000000e-07,106
107,0.107684,0.963266,0.436422,0.8879,1.000000e-07,107
108,0.107050,0.963507,0.436347,0.8879,1.000000e-07,108


In [27]:
test_loss, test_acc = model_dn201.evaluate(x_test_norm,  y_test10, verbose=0)
print('Accuracy on test data:', test_acc)

Accuracy on test data: 0.8823999762535095


In [70]:
# ResNet50
imp_net2 = keras.applications.ResNet50(weights='imagenet',include_top=False,
            input_shape=(32, 32, 3))

In [71]:
model_resnet = Sequential()
model_resnet.add(imp_net2)
model_resnet.add(Dropout(0.25))
model_resnet.add(Flatten())
model_resnet.add(Dense(512, activation='relu'))
model_resnet.add(Dense(128, activation='relu'))
model_resnet.add(Dropout(0.10))
model_resnet.add(Dense(10, activation='softmax'))
model_resnet.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.0001), metrics=["accuracy"])
model_resnet.summary()

imp_net2.trainable = False #Freezing weights

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 1, 1, 2048)        23587712  
_________________________________________________________________
dropout_21 (Dropout)         (None, 1, 1, 2048)        0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 2048)              0         
_________________________________________________________________
dense_31 (Dense)             (None, 512)               1049088   
_________________________________________________________________
dense_32 (Dense)             (None, 128)               65664     
_________________________________________________________________
dropout_22 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_33 (Dense)             (None, 10)              

In [72]:
fw2 = go.FigureWidget()
fw2

FigureWidget({
    'data': [], 'layout': {'template': '...'}
})

In [73]:
batch_size=128
callbacks_list=[learn_rate,early_stop,acc_mon(fw2)]
report=model_resnet.fit(datagen.flow(x_train_norm, y_train10, batch_size=batch_size),\
                    steps_per_epoch=x_train_norm.shape[0] // batch_size,epochs=1000,\
                    verbose=0,validation_data=(x_test_norm,y_test10),callbacks=callbacks_list)


Start training:
..........-> epoch: 10  loss : 0.55303663  accuracy : 0.81252003  val_loss : 0.57065111  val_accuracy : 0.80980003  lr : 1e-04  
..........-> epoch: 20  loss : 0.39386395  accuracy : 0.86547559  val_loss : 0.50730085  val_accuracy : 0.83450001  lr : 1e-04  
..........-> epoch: 30  loss : 0.29635283  accuracy : 0.89655519  val_loss : 0.55135411  val_accuracy : 0.83179998  lr : 1e-04  
..........-> epoch: 40  loss : 0.2366147  accuracy : 0.91628569  val_loss : 0.54772902  val_accuracy : 0.84469998  lr : 1e-04  
..........-> epoch: 50  loss : 0.13773252  accuracy : 0.95297962  val_loss : 0.54652596  val_accuracy : 0.85930002  lr : 1e-05  
..........-> epoch: 60  loss : 0.10654998  accuracy : 0.96234357  val_loss : 0.57273352  val_accuracy : 0.86089998  lr : 1e-05  
..........-> epoch: 70  loss : 0.09776971  accuracy : 0.96601301  val_loss : 0.57503802  val_accuracy : 0.86299998  lr : 1e-06  
..........-> epoch: 80  loss : 0.09479134  accuracy : 0.9664942  val_loss : 0.5773

In [74]:
df_report = pd.DataFrame(report.history) 
df_report['epoch'] = report.epoch
df_report.tail()

,loss,accuracy,val_loss,val_accuracy,lr,epoch
77,0.099461,0.965913,0.576102,0.8635,1.000000e-07,77
78,0.096712,0.966434,0.578117,0.8635,1.000000e-07,78
79,0.098295,0.965492,0.578001,0.8634,1.000000e-07,79
80,0.094791,0.966494,0.577301,0.8626,1.000000e-07,80
81,0.094394,0.967296,0.575876,0.8628,1.000000e-07,81


In [75]:
test_loss, test_acc = model_resnet.evaluate(x_test_norm,  y_test10, verbose=0)
print('Accuracy on test data:', test_acc)

Accuracy on test data: 0.8501999974250793


In [46]:
# ResNet101
imp_net3 = keras.applications.ResNet101(weights='imagenet',include_top=False,
            input_shape=(32, 32, 3))

In [47]:
model_r101 = Sequential()
model_r101.add(imp_net3)
model_r101.add(Dropout(0.25))
model_r101.add(Flatten())
model_r101.add(Dense(512, activation='relu'))
model_r101.add(Dense(128, activation='relu'))
model_r101.add(Dropout(0.10))
model_r101.add(Dense(10, activation='softmax'))
model_r101.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.001), metrics=["accuracy"])
model_r101.summary()

imp_net3.trainable = False #Freezing weights

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet101 (Functional)       (None, 1, 1, 2048)        42658176  
_________________________________________________________________
dropout_20 (Dropout)         (None, 1, 1, 2048)        0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 2048)              0         
_________________________________________________________________
dense_30 (Dense)             (None, 512)               1049088   
_________________________________________________________________
dense_31 (Dense)             (None, 128)               65664     
_________________________________________________________________
dropout_21 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_32 (Dense)             (None, 10)              

In [20]:
fw3 = go.FigureWidget()
fw3

FigureWidget({
    'data': [], 'layout': {'template': '...'}
})

In [48]:
callbacks_list=[learn_rate,early_stop,acc_mon(fw3,20,0.5)]
batch_size=128
report=model_r101.fit(datagen.flow(x_train_norm, y_train10, batch_size=batch_size),\
                    steps_per_epoch=x_train_norm.shape[0] // batch_size,epochs=1000,\
                    verbose=0,validation_data=(x_test_norm,y_test10),callbacks=callbacks_list)

Start training:
..........-> epoch: 10  loss : 0.96405256  accuracy : 0.67194015  val_loss : 1.14673066  val_accuracy : 0.64770001  lr : 0.001  
..........-> epoch: 20  loss : 0.91897351  accuracy : 0.68487328  val_loss : 1.79563379  val_accuracy : 0.39359999  lr : 0.001  
..........-> epoch: 30  loss : 0.65400273  accuracy : 0.78081888  val_loss : 0.77368802  val_accuracy : 0.75220001  lr : 0.001  
..........-> epoch: 40  loss : 0.57820773  accuracy : 0.80668509  val_loss : 0.64848596  val_accuracy : 0.78289998  lr : 0.001  
..........-> epoch: 50  loss : 0.49918064  accuracy : 0.83192974  val_loss : 0.78458899  val_accuracy : 0.74769998  lr : 0.001  
..........-> epoch: 60  loss : 0.4314962  accuracy : 0.85396618  val_loss : 0.62284517  val_accuracy : 0.80510002  lr : 0.001  
..........-> epoch: 70  loss : 0.43055674  accuracy : 0.85542989  val_loss : 0.50128996  val_accuracy : 0.83380002  lr : 0.001  
..........-> epoch: 80  loss : 0.35604373  accuracy : 0.88075471  val_loss : 0.934

In [49]:
df_report = pd.DataFrame(report.history) 
df_report['epoch'] = report.epoch
df_report.tail()

,loss,accuracy,val_loss,val_accuracy,lr,epoch
144,0.196096,0.933289,0.456333,0.8665,1.000000e-07,144
145,0.195572,0.931190,0.457700,0.8669,1.000000e-07,145
146,0.192965,0.933008,0.456976,0.8673,1.000000e-07,146
147,0.196684,0.931144,0.456562,0.8670,1.000000e-07,147
148,0.196076,0.931043,0.457054,0.8667,1.000000e-07,148


In [50]:
test_loss, test_acc = model_r101.evaluate(x_test_norm,  y_test10, verbose=0)
print('Accuracy on test data:', test_acc)

Accuracy on test data: 0.8650000095367432


In [14]:
# DenseNet 169
imp_net4 = keras.applications.DenseNet169(weights='imagenet',include_top=False,
            input_shape=(32, 32, 3))

In [15]:

model_dn169 = Sequential()
model_dn169.add(imp_net4)
model_dn169.add(Dropout(0.25))
model_dn169.add(Flatten())
model_dn169.add(Dense(416, activation='relu'))
model_dn169.add(Dense(104, activation='relu'))
model_dn169.add(Dense(26, activation='relu'))
model_dn169.add(Dropout(0.10))
model_dn169.add(Dense(10, activation='softmax'))
model_dn169.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
imp_net4.trainable = False #Freezing weights
model_dn169.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet169 (Functional)     (None, 1, 1, 1664)        12642880  
_________________________________________________________________
dropout (Dropout)            (None, 1, 1, 1664)        0         
_________________________________________________________________
flatten (Flatten)            (None, 1664)              0         
_________________________________________________________________
dense (Dense)                (None, 416)               692640    
_________________________________________________________________
dense_1 (Dense)              (None, 104)               43368     
_________________________________________________________________
dense_2 (Dense)              (None, 26)                2730      
_________________________________________________________________
dropout_1 (Dropout)          (None, 26)                0

In [16]:
fw4 = go.FigureWidget()
fw4

FigureWidget({
    'data': [], 'layout': {'template': '...'}
})

In [17]:
callbacks_list=[learn_rate,early_stop,acc_mon(fw4,20,0.5)]
batch_size=128
report=model_dn169.fit(datagen.flow(x_train_norm, y_train10, batch_size=batch_size),\
                    steps_per_epoch=x_train_norm.shape[0] // batch_size,epochs=1000,\
                    verbose=0,validation_data=(x_test_norm,y_test10),callbacks=callbacks_list)

Start training:
..........-> epoch: 10  loss : 0.6079585  accuracy : 0.80004811  val_loss : 1.06246078  val_accuracy : 0.68519998  lr : 0.001  
..........-> epoch: 20  loss : 0.45727929  accuracy : 0.84779036  val_loss : 0.66080946  val_accuracy : 0.80360001  lr : 0.001  
..........-> epoch: 30  loss : 0.3747499  accuracy : 0.87572187  val_loss : 0.53169459  val_accuracy : 0.83859998  lr : 0.001  
..........-> epoch: 40  loss : 0.32686201  accuracy : 0.89033926  val_loss : 0.85940361  val_accuracy : 0.74980003  lr : 0.001  
..........-> epoch: 50  loss : 0.27237016  accuracy : 0.90944821  val_loss : 0.57014853  val_accuracy : 0.82569999  lr : 0.001  
..........-> epoch: 60  loss : 0.15010747  accuracy : 0.94983155  val_loss : 0.42537475  val_accuracy : 0.88069999  lr : 1e-04  
..........-> epoch: 70  loss : 0.12112873  accuracy : 0.95947623  val_loss : 0.46252075  val_accuracy : 0.87819999  lr : 1e-04  
..........-> epoch: 80  loss : 0.10487733  accuracy : 0.9639076  val_loss : 0.59155

In [18]:
df_report = pd.DataFrame(report.history) 
df_report['epoch'] = report.epoch
df_report.tail()

,loss,accuracy,val_loss,val_accuracy,lr,epoch
113,0.093332,0.968138,0.538536,0.8907,1.000000e-07,113
114,0.090920,0.969823,0.572157,0.8909,1.000000e-07,114
115,0.092387,0.968700,0.674627,0.8898,1.000000e-07,115
116,0.087618,0.970464,0.542723,0.8913,1.000000e-07,116
117,0.087953,0.969562,0.556804,0.8916,1.000000e-07,117


In [19]:
test_loss, test_acc = model_dn169.evaluate(x_test_norm,  y_test10, verbose=0)
print('Accuracy on test data:', test_acc)

Accuracy on test data: 0.8949000239372253
